In [2]:
import pdb
import sys
sys.path = ['/nfs/gns/homes/willj/anaconda3/envs/GTEx/lib/python3.5/site-packages'] + sys.path
import pickle
import math
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.stats import pearsonr
from lim.genetics import qtl
from lim.genetics.phenotype import NormalPhenotype
import logging
import argparse
import os
from pyensembl import EnsemblRelease
lim_logging = logging.getLogger('lim')
lim_logging.setLevel(logging.CRITICAL)
import h5py

In [4]:
GTEx_directory = '/hps/nobackup/research/stegle/users/willj/GTEx'

In [26]:
tissue = 'Lung'
tissue_expression_filepath = '/nfs/research2/stegle/stegle_secure/GTEx/download/49139/PhenoGenotypeFiles/RootStudyConsentSet_phs000424.GTEx.v6.p1.c1.GRU/ExpressionFiles/phe000006.v2.GTEx_RNAseq.expression-data-matrixfmt.c1/parse_data/44_tissues/GTEx_Data_20150112_RNAseq_RNASeQCv1.1.8_gene_rpkm_{}_normalised_without_inverse_gene_expression.txt'.format(tissue)
with open(tissue_expression_filepath, 'r') as f:
    expression_table = np.array([x.split('\t') for x in f.read().splitlines()])
    expression_matrix = expression_table[1:,1:].astype(np.float32)

individual_tissue_expression_donor_IDs = [x.split('-')[1] for x in expression_table[0,:][1:]]
expression_IDs = [n.encode("ascii", "ignore") for n in expression_table[1:,0]]

In [27]:
with h5py.File(GTEx_directory + '/retrained_inceptionet_aggregations.hdf5','w') as g:
    with h5py.File(GTEx_directory + '/retrained_inceptionet_features.hdf5','r') as f:
        for size in list(f['lung']['features'].keys()):
            mean_aggregated_features = []
            median_aggregated_features = []
            aggregated_expression = []
            print (size)
            for ID in list(f['lung']['features'][size].keys()):
                patch_features = f['lung']['features'][size][ID].value
                mean_patch_features = np.mean(patch_features,axis=0)
                median_patch_features = np.median(patch_features,axis=0)
                
                mean_aggregated_features.append(mean_patch_features)
                median_aggregated_features.append(median_patch_features)
            
                image_expression = f['lung']['expression'][ID].value
                aggregated_expression.append(image_expression)
                
            size_group = g.create_group('lung/{}'.format(size))
            size_group.create_dataset('median', data=mean_aggregated_features)
            size_group.create_dataset('mean', data=median_aggregated_features)
            size_group.create_dataset('expression',data=aggregated_expression)
            size_group.create_dataset('expression_IDs',data=expression_IDs)
    



1024
128
2048
256
512


In [32]:
with h5py.File(GTEx_directory + '/retrained_inceptionet_aggregations.hdf5','r') as g:
    print (g['lung']['256']['expression'].value.shape)

(274, 28683)
